# Sync overview

Feature layers includes a `Sync` capability, which when enabled, allows client applications to take feature layers offline, perform edits and sync it back to the layer. When you checkout some features and store it offline in the client, you call that a `replica`. The `FeatureLayerCollection` class under the `features` module allows users to create and work with replicas. The workflow of using sync involves these three operations:
- Create replica
- Synchronize replica
- Unregister replica

To learn more about this feature, refer to the [documentation here](http://resources.arcgis.com/en/help/arcgis-rest-api/index.html#//02r300000112000000)

## Checking out data from feature layers using replicas
To create a replica, we need a feature layer that is sync enabled. We can use the `syncEnabled` property of a `FeatureLayer` object to verify that. Further the `syncCapabilities` property returns a dictionary with fine grained sync capabilities

In [1]:
# connect to a GIS
from arcgis.gis import GIS
import arcgis.features
gis = GIS() # connect to www.arcgis.com anonymously. 
            # we will use a public sync enabled feature layer

To create and work with replicas, we need to create a `FeatureLayerCollection` object. A `FeatureLayerCollection` object can be created from either a feature layer `Item` or directly using a feature service URL.

Here, we will connect to a sample service by esri with URL `https://sampleserver6.arcgisonline.com/arcgis/rest/services/Sync/WildfireSync/FeatureServer/` 

In [2]:
url = 'https://sampleserver6.arcgisonline.com/arcgis/rest/services/Sync/WildfireSync/FeatureServer/'
wildfire_flc = arcgis.features.FeatureLayerCollection(url, gis)

In [3]:
type(wildfire_flc)

arcgis.features.layer.FeatureLayerCollection

### Verify if sync is enabled

Accessing the `layers` property on a `FeatureLayerCollection` returns a list of `FeatureLayer` objects. We can create a replica of one of these or all of these layers

In [4]:
wildfire_flc.layers

[<FeatureLayer url:"https://sampleserver6.arcgisonline.com/arcgis/rest/services/Sync/WildfireSync/FeatureServer//0">,
 <FeatureLayer url:"https://sampleserver6.arcgisonline.com/arcgis/rest/services/Sync/WildfireSync/FeatureServer//1">,
 <FeatureLayer url:"https://sampleserver6.arcgisonline.com/arcgis/rest/services/Sync/WildfireSync/FeatureServer//2">]

In [5]:
# query syncEnabled property to verify is sync is enabled
wildfire_flc.properties.syncEnabled

True

In [6]:
# query the syncCapabilities property to view fine grained capabilities
wildfire_flc.properties.syncCapabilities

{
  "supportsRegisteringExistingData": true,
  "supportsSyncDirectionControl": true,
  "supportsPerLayerSync": true,
  "supportsPerReplicaSync": false,
  "supportsRollbackOnFailure": false,
  "supportsAsync": true,
  "supportsAttachmentsSyncDirection": true,
  "supportsSyncModelNone": true
}

### List existing replicas

The `replicas` property on a `FeatureLayerCollection` object returns you a `SyncManager` object. You would work with this manager object for all your sync workflows.

You can find if any replicas were created earlier on this layer by calling `get_list()` method on the `SyncManager` object.

In [7]:
replica_list = wildfire_flc.replicas.get_list()
len(replica_list)

2180

As you can see, there are plenty of replicas on this layer. Let us view one of it

In [8]:
replica_list[0]

{'replicaName': 'Ags_Fs_1543141555952',
 'replicaID': 'A2A63918-F35D-4441-8AE8-693A672F4BED'}

### Create a replica
Now, let us create our own replica of this feature layer. The `create()` method accepts a number of parameters allowing you to adjust what needs to be replicated and customize other options. For more information on this operation, refer to the [documentation here](http://resources.arcgis.com/en/help/arcgis-rest-api/index.html#//02r3000000rp000000).

The full capability of the sync operation allows you to check out data from a feature layer, make edits and sync the deltas (changes) back to the server and update the features. This is popular in use cases which involve client applications such as ArcGIS Runtime or ArcGIS Desktop applications check out data, go offline (such as in areas where network connectivity is limited), make edits, then synchronize the data back to the server and update the features. The capability allows multiple clients to do this in parallel, thus enabling a large data collection effort.

However, if your purpose of a replica is only to check out the data (one directional), then you can verify if the `extract` capability is enabled on the feature layer and create a replica that is just meant for data check out. We will see this use case below:

In [9]:
# list all capabilities
wildfire_flc.properties.capabilities

'Create,Delete,Query,Sync,Update,Uploads,Editing'

This layer has disabled 'Extract'. Hence let us search for a different layer

In [10]:
portal_gis = GIS("portal url", "username", "password")
search_result = portal_gis.content.search("Chennai Precipitation", "Feature Layer")

In [ ]:
search_result[0]

Let us create a `FeatureLayerCollection` object from this item

In [12]:
precip_flc = arcgis.features.FeatureLayerCollection.fromitem(search_result[0])
type(precip_flc)

arcgis.features.layer.FeatureLayerCollection

#### Verify Extract capability

In [13]:
precip_flc.properties.capabilities

'Create,Editing,Query,Update,Uploads,Delete,Sync,Extract'

This is a suitable feature layer (since `Sync` and `Extract` are both enabled as seen in the capabilities), let us extract the data into a file geodatabase and store it in local file system

In [14]:
precip_flc = arcgis.features.FeatureLayerCollection.fromitem(search_result[0])

The "fullExtent" of the speicifc `FeatureLayerCollection` object can be extracted and used to construct the geometry filter to be used in the `create` method mentioned later.

In [24]:
extents =precip_flc.properties['fullExtent']
extents_str = ",".join(format(x, "10.3f") for x in [extents['xmin'],extents['ymin'],extents['xmax'], extents['ymax']])
extents_str

'8608022.302,1006191.251,8937015.925,1498443.166'

Now we are ready to create the geometry filter. `geom_filter` is a spatial filter class object from arcgis.geometry.filters module to filter results by a spatial relationship with geometry (in this case, using the full extents of the AOI)

In [25]:
geom_filter = {'geometryType':'esriGeometryEnvelope'}
geom_filter.update({'geometry':extents_str})
geom_filter

{'geometryType': 'esriGeometryEnvelope',
 'geometry': '8608022.302,1006191.251,8937015.925,1498443.166'}

In [27]:
replica1 = precip_flc.replicas.create(replica_name = 'arcgis_python_api_precip',
                                      layers='0',
                                      geometry_filter=geom_filter,
                                      sync_model="perLayer",
                                      target_type="server",
                                      data_format="sqlite",
                                      out_path=r'/arcgis/home')
replica1

{'transportType': 'esriTransportTypeURL',
 'replicaName': 'arcgis_python_1553302281853',
 'responseType': 'esriReplicaResponseTypeData',
 'replicaID': 'B209D686-DD3F-4DD4-801A-99D44169A73D',
 'targetType': 'server',
 'layerServerGens': [{'id': 0, 'serverGen': 1553302280841, 'serverSibGen': 0}],
 'resultUrl': 'https://datascienceqa.esri.com/server/rest/directories/arcgisjobs/system/synctoolsex_gpserver/ja8cc9fd202174283ba2063754bcc897d/scratch/_ags_data57934B245C4640938213458F31E42C90.geodatabase',
 'submissionTime': 1553302280000,
 'lastUpdatedTime': 1553302282000,
 'status': 'Completed'}

Thus, we were able to checkout data from this feature layer into a file geodatabase. Clients can use this data in any way they wish, for instance, publish it as another feature layer to a different portal or just store it for archival.

## Removing replicas

The sync operation is expensive on the resources of your web GIS. Hence, it is a good maintenance practice to remove unnecessary replicas. An ArcGIS admin could use the ArcGIS Python API to script and automate the process of scanning all feature layers and removing stale replicas on each of them.

A replica can be removed by calling the `unregister()` method and passing the id of a replica that needs to be removed.

In [28]:
# Let us query all the replicas registered on the ports feature layer from before
replica_list = precip_flc.replicas.get_list()

In [29]:
for r in replica_list:
    print(r)

{'replicaName': 'arcgis_python_1553302281853', 'replicaID': 'B209D686-DD3F-4DD4-801A-99D44169A73D'}
{'replicaName': 'arcgis_python_api_precip', 'replicaID': 'A32B2251-B916-4A5B-8CCF-AD38EB3DAEDE'}


There are more than a few. I only want to remove the replicas that were registered 10 minutes ago. But, your search criteria could be any other. 

We will loop through each of the replicas returned and use the `get()` method to get detailed information about these replicas and look a `creationDate` property.

Before looping, let us take a deeper look at one of these replicas by calling the `get()` method:

In [30]:
replica1 = precip_flc.replicas.get(replica_list[0]['replicaID'])
replica1

{'replicaName': 'arcgis_python_1553302281853',
 'replicaID': 'B209D686-DD3F-4DD4-801A-99D44169A73D',
 'replicaOwner': 'portaladmin',
 'spatialReference': {'wkid': 102100, 'latestWkid': 3857},
 'layerServerGens': [{'id': 0, 'serverGen': 1553302280841, 'serverSibGen': 0}],
 'creationDate': 1553302280841,
 'lastSyncDate': 1553302280841,
 'syncModel': 'perLayer',
 'targetType': 'server',
 'syncDirection': 'download',
 'returnsAttachments': False,
 'returnAttachments': False,
 'attachmentsSyncDirection': 'none',
 'spatialRel': 'esriSpatialRelIntersects',
 'geometry': {'xmin': 8608022.302,
  'ymin': 1006191.251,
  'xmax': 8937015.925,
  'ymax': 1498443.166,
  'spatialReference': {'wkid': 102100, 'latestWkid': 3857}},
 'layers': [{'id': 0,
   'queryOption': 'useFilter',
   'useGeometry': True,
   'includeRelated': True,
   'where': ''}]}

The `creationDate` key is retured as unix epoch time. We need to convert it to local time for processing:

In [31]:
import time
time.localtime(replica1['creationDate']/1000) #dividing by 1000 to convert micro seconds to seconds

time.struct_time(tm_year=2019, tm_mon=3, tm_mday=22, tm_hour=17, tm_min=51, tm_sec=20, tm_wday=4, tm_yday=81, tm_isdst=1)

To determine those replicas that were created 10 mins earlier, let us create an epoch timestamp for 10 mins before now and find those replicas whose time stamps are lower than this

In [32]:
ten_min_earlier_epoch = time.time() - 10
ten_min_earlier_epoch

1553302277.890686

In [33]:
import time
removal_list = []
for r in replica_list:
    temp_r = precip_flc.replicas.get(r['replicaID'])
    temp_dict = {'replica_id': r['replicaID'],
                'creationDate':temp_r['creationDate']/1000}
    
    #check
    if temp_dict['creationDate'] < ten_min_earlier_epoch:
        removal_list.append(temp_dict)
        print(temp_dict)

{'replica_id': 'A32B2251-B916-4A5B-8CCF-AD38EB3DAEDE', 'creationDate': 1553302241.936}


Let us loop through each of these replicas and remove them using the `unregister()` method:

In [34]:
for r in removal_list:
    result = precip_flc.replicas.unregister(r['replica_id'])
    print(result)

{'success': True}
